# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. 

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

**Цель исследования:** Построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализировать возможную прибыль и риски техникой *Bootstrap*.

**Ход исследования**

Исследование пройдёт в 4 этапa:

  - Загрузка и подготовка данных, поиск аномалий и дубликатов в данных;
  - Обучение и проверка модели
  - Подготовка к расчету прибыли
  - Расчет прибыли и рисков

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState

In [2]:
try:
    geo_0=pd.read_csv('/datasets/geo_data_0.csv')
    geo_1=pd.read_csv('/datasets/geo_data_1.csv')
    geo_2=pd.read_csv('/datasets/geo_data_2.csv')
except:
    geo_0=pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_0.csv')
    geo_1=pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_1.csv')
    geo_2=pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_2.csv')

In [3]:
geo_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
geo_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
geo_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [6]:
geo_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [7]:
geo_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [8]:
geo_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


**Описание данных**

В таблицах приведены данные геологоразведки трёх регионов, в каждой по 5 колонок типа -float64- и -object-.

**Признаки**

* `id` - уникальный идентификатор скважины;
* `f0`, `f1`, `f2` - значимые признаки.
* `product` - объём запасов в скважине (тыс. баррелей).

In [9]:
geo_0.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [10]:
geo_1.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [11]:
geo_2.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

Пропусков в данных нет, проверим их на наличие дубликатов и избавимся от аномалий.

### Aномалии, дубликаты

**Рассмотрим каждую таблицу. Начнем с первого региона:**

In [12]:
geo_0['id'].nunique()

99990

In [13]:
geo_0['id'].value_counts(ascending=False)

QcMuo    2
bsk9y    2
A5aEY    2
fiKDv    2
HZww2    2
        ..
qJEEl    1
NPqf3    1
csZvJ    1
v558w    1
E6v56    1
Name: id, Length: 99990, dtype: int64

In [14]:
geo_0[geo_0['id'] == 'AGS9W']

,id,f0,f1,f2,product
42529,AGS9W,1.454747,-0.479651,0.683380,126.370504
69163,AGS9W,-0.933795,0.116194,-3.655896,19.230453


В данной таблице есть дубликаты  `id`, удалим все вхождения, кроме первого.

In [15]:
geo_0['id'] = geo_0['id'].drop_duplicates(keep='first')
geo_0 = geo_0.dropna(subset=['id']).reset_index(drop=True)

In [16]:
geo_0.describe()

,f0,f1,f2,product
count,99990.000000,99990.000000,99990.000000,99990.000000
mean,0.500454,0.250141,2.502629,92.499684
std,0.871844,0.504430,3.248149,44.288304
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072572,-0.200877,0.287784,56.497069
50%,0.502405,0.250252,2.515969,91.847928
75%,1.073626,0.700640,4.715035,128.563699
max,2.362331,1.343769,16.003790,185.364347


In [17]:
geo_0.corr()

,f0,f1,f2,product
f0,1.000000,-0.440717,-0.003211,0.143504
f1,-0.440717,1.000000,0.001764,-0.192351
f2,-0.003211,0.001764,1.000000,0.483631
product,0.143504,-0.192351,0.483631,1.000000


Корреляция объема запасов и признаков *f0* и *f1* отсутствует. C признаком *f2*, чьи значения имеют наибольший разброс, слабая положительная корреляция.

In [18]:
geo_0.duplicated().sum()

0

**Рассмотрим следующий регион:**

In [19]:
geo_1['id'].nunique()

99996

In [20]:
geo_1['id'].value_counts(ascending=False)

wt4Uk    2
bfPNe    2
5ltQ6    2
LHZR0    2
5Iyzi    1
        ..
sUvmI    1
ivIzf    1
Ubnvb    1
HssUH    1
pcfAn    1
Name: id, Length: 99996, dtype: int64

В данной таблице есть дубликаты  `id`, удалим все вхождения, кроме первого.

In [21]:
geo_1['id'] = geo_1['id'].drop_duplicates(keep='first')
geo_1 = geo_1.dropna(subset=['id']).reset_index(drop=True)

In [22]:
geo_1.duplicated().sum()

0

In [23]:
geo_1.describe()

,f0,f1,f2,product
count,99996.000000,99996.000000,99996.000000,99996.000000
mean,1.141209,-4.796608,2.494501,68.823916
std,8.965815,5.119906,1.703579,45.944663
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011475,57.085625
75%,8.620964,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [24]:
geo_1.corr()

,f0,f1,f2,product
f0,1.000000,0.182279,-0.001810,-0.030523
f1,0.182279,1.000000,-0.002594,-0.010154
f2,-0.001810,-0.002594,1.000000,0.999397
product,-0.030523,-0.010154,0.999397,1.000000


В этом регионе объем запасов сильно зависит от признака *f2*, с остальными связь отсутствует. У признаков *f0*, *f1* больший разброс значений.

**Рассмотрим последний регион:**

In [25]:
geo_2['id'].nunique()

99996

In [26]:
geo_2['id'].value_counts(ascending=False)

xCHr8    2
KUPhW    2
VF7Jo    2
Vcm5J    2
pgfzQ    1
        ..
fk6Zt    1
3UA68    1
N3hxr    1
Sbw8P    1
lIEiW    1
Name: id, Length: 99996, dtype: int64

В данной таблице есть дубликаты  `id`, удалим все вхождения, кроме первого.

In [27]:
geo_2['id'] = geo_2['id'].drop_duplicates(keep='first')
geo_2 = geo_2.dropna(subset=['id']).reset_index(drop=True)

In [28]:
geo_2.duplicated().sum()

0

In [29]:
geo_2.describe()

,f0,f1,f2,product
count,99996.000000,99996.000000,99996.000000,99996.000000
mean,0.002002,-0.002159,2.495084,94.998342
std,1.732052,1.730397,3.473482,44.749573
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162328,-1.174841,0.130269,59.450028
50%,0.009424,-0.009661,2.484236,94.925026
75%,1.158477,1.163523,4.858720,130.586815
max,7.238262,7.844801,16.739402,190.029838


In [30]:
geo_2.corr()

,f0,f1,f2,product
f0,1.000000,0.000506,-0.000452,-0.001978
f1,0.000506,1.000000,0.000753,-0.001055
f2,-0.000452,0.000753,1.000000,0.445867
product,-0.001978,-0.001055,0.445867,1.000000


В данном регионе, как и в остальных наблюдается слабая корреляция с признаком *f2*, имеющим наибольший разброс. Удалим в этом столбце сильно выбивающиеся значения:

***Вывод***

На данном этапе удалены повторяющиеся номера месторождений.
Явных дубликатов в данных нет, наибольшая корреляция во всех регионах имеется с признаком *f2*.

## Обучение и проверка модели

Обучим модель Линейной регрессии на имеющихся данных. Необходимо маштабирование признаков `f0`, `f1`, `f2`

In [31]:
# Подготовка данных:
def spliting(row):
    target = row['product']
    features = row.drop(['product', 'id'] , axis=1)
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=12345)
    return features_train, features_valid, target_train, target_valid
    
def const_model(row):
    features_train, features_valid, target_train, target_valid = spliting(row)
    predicted_valid_const = pd.Series(target_train.median(), index=target_valid.index)
    error = mean_squared_error(target_valid, predicted_valid_const)**0.5
    print(f'Ошибка константной модели: {error}')
    return error

def scalaring(row):
    features_train, features_valid, target_train, target_valid = spliting(row)
    scaler = StandardScaler()
    scaler.fit(features_train)
    features_train = scaler.transform(features_train)
    features_valid = scaler.transform(features_valid)
    return features_train, features_valid, target_train, target_valid

def fit_predict(row):
    features_train, features_valid, target_train, target_valid = scalaring(row)
    model = LinearRegression()
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    return predicted_valid, model

reserve_all = []
def metrics(row):
    features_train, features_valid, target_train, target_valid = scalaring(row)
    predicted_valid, model = fit_predict(row)
    
    mean_reserve = pd.Series(predicted_valid).mean()
    reserve_all.append(mean_reserve)
    rmse = mean_squared_error(target_valid, pd.Series(predicted_valid))** 0.5
    print('Cредний запас предсказанного сырья:', mean_reserve)
    print('rmse:', rmse)
    print('-----------')
    if rmse < const_model(row):
        print('Данная модель лучше константной')
    else:
        print('Данная модель не адекватна')
    return mean_reserve, rmse

**РЕГИОНЫ** Рассмотрим каждый регион и предсказания по модели:

In [32]:
# Предсказания для каждого региона
p_valid_0, _ = fit_predict(geo_0)
p_valid_1, _ = fit_predict(geo_1)
p_valid_2, _ = fit_predict(geo_2)

In [33]:
region_0 = metrics(geo_0)

Cредний запас предсказанного сырья: 92.78915638280621
rmse: 37.853527328872964
-----------
Ошибка константной модели: 44.37708333688604
Данная модель лучше константной


In [34]:
region_1 = metrics(geo_1)

Cредний запас предсказанного сырья: 69.1783195703043
rmse: 0.8920592647717033
-----------
Ошибка константной модели: 47.46890162326147
Данная модель лучше константной


In [35]:
region_2 = metrics(geo_2)

Cредний запас предсказанного сырья: 94.86572480562035
rmse: 40.07585073246016
-----------
Ошибка константной модели: 44.68304659574556
Данная модель лучше константной


***Вывод***

Согласно данным, каждая обученная модель показала себя лучше константной, при этом:

* наибольший средний запас предсказанного сырья достигнут во 2-м Регионе;
* лучше всего модель работает по данным Региона №1, где наименьшее значение rmse и сильной корреляции с признаком *f2*

## Подготовка к расчёту прибыли

In [36]:
# значения для расчётов (в миллионах)
BEST_COUNT = 200
N = 500
INCOME = 0.45
BUDGET = 10**4
LOSS_PR = 0.025

In [37]:
breakeven_reserve = round(BUDGET/(BEST_COUNT*INCOME), 2)
print(f'Oбъём сырья для безубыточной разработки составил {breakeven_reserve} баррель(ей)')

Oбъём сырья для безубыточной разработки составил 111.11 баррель(ей)


In [38]:
# средние запасы по регионам
reserve_all

[92.78915638280621, 69.1783195703043, 94.86572480562035]

Итак, средний запас предсказанного сырья в каждом регионе не превышает уровня безубыточной разработки. Это означает, что половина всех месторождений Регионов в перспективе убыточна. Рассмотри далее, найдутся ли 200 точек, способных обеспечить потенциальную прибыль.

## Расчёт прибыли и рисков 

In [39]:
# функция для расчёта прибыли по выбранным скважинам
def profit(row, predictions):
    count = BEST_COUNT
    _,_,_, target_valid = spliting(row)
    p_sorted = pd.Series(predictions).sort_values(ascending=False)[:count]
    selected = target_valid.reset_index(drop=True)[p_sorted.index]
    region_revenue = round((selected.sum()*INCOME - BUDGET), 3)
    return region_revenue

In [40]:
reg_0_income = profit(geo_0, p_valid_0)
print(f'Прибыль двухсот лучших скважин данного Региона №0 составила {reg_0_income} млн.руб.')

Прибыль двухсот лучших скважин данного Региона №0 составила 3365.187 млн.руб.


In [41]:
reg_1_income = profit(geo_1, p_valid_1)
print(f'Прибыль двухсот лучших скважин данного Региона №1 составила {reg_1_income} млн.руб.')

Прибыль двухсот лучших скважин данного Региона №1 составила 2415.087 млн.руб.


In [42]:
reg_2_income = profit(geo_2, p_valid_2)
print(f'Прибыль двухсот лучших скважин данного Региона №2 составила {reg_2_income} млн.руб.')

Прибыль двухсот лучших скважин данного Региона №2 составила 2501.284 млн.руб.


Согласно модели, наибольшую прибыль с двухсот лучших скважин можно достич в Регионе №0. Определим далее 95%-й доверительный интервал:

In [47]:
state = RandomState(12345)
def confidence_interval(row, predictions):
    values = []
    for i in range(1000):
        predict_subs = pd.Series(predictions).sample(N, replace=True, random_state=state)
        values.append(profit(row, predict_subs))
    
    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    risks = (values < 0).mean()
    mean = values.mean()
    print(f"Средняя выручка: {round(mean, 3)} млн.руб")
    print(f'95% доверительный интервал: ({round(lower,3)}, {round(upper,3)})')
    print(f'Риск убытков составил {risks * 100}%')
    if risks > LOSS_PR:
        print('---Вероятность убытков высока!---')
    else:
        print('Регион подходит для разработки!')
    return lower, upper

In [44]:
interval_0 = confidence_interval(geo_0, p_valid_0)

Средняя выручка: 380.613 млн.руб
95% доверительный интервал: (-142.942, 890.977)
Риск убытков составил 7.7%
---Вероятность убытков высока!---


In [48]:
interval_1 = confidence_interval(geo_1, p_valid_1)

Средняя выручка: 478.485 млн.руб
95% доверительный интервал: (89.525, 866.604)
Риск убытков составил 1.2%
Регион подходит для разработки!


In [46]:
interval_2 = confidence_interval(geo_2, p_valid_2)

Средняя выручка: 315.442 млн.руб
95% доверительный интервал: (-216.265, 811.061)
Риск убытков составил 12.4%
---Вероятность убытков высока!---


Итак, можно говорить, что добыча в Регионах №0 и №2 потенциально убыточна, риски превышают уровень 2,5%. Наименьший риск убытков(1,2%) наблюдается в Регионе №1, где 1) средняя выручка составила 478,485 млн. рублей; 2) с вероятностью 95% истинные значения прибыли находятся в интервале (89.525, 866.604)

**Общий вывод**

Обучив и проверив модель Линейной регрессии на данных о добыче нефти в трех регионах было установлено следующее:

- средний запас предсказанного сырья в каждом регионе не превышает уровня безубыточной разработки; наибольшее среднее достигнуто в Регионе №2
- лучше всего модель работает по данным Региона №1, где наименьшее значение rmse
- Добыча в Регионе №1 потенциально прибыльна, риски не превышают уровень 2,5%. Поэтому только его можно рекомендовать для разработки.